In [1]:
#Standard python libraries
import os
import time
import logging
import pickle
 
# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
import torch

#sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\train.csv")

In [3]:
df = df.drop(columns=["Unnamed: 0"])
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0.0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1.0
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1.0
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1.0
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0.0
61996,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1.0
61997,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0.0
61998,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1.0


In [4]:
df.describe()

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
count,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,...,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000,62000.000000
mean,-11.724671,-17.971911,2036.387745,2055.863785,2249.486666,2608.814367,2978.149583,3128.583860,3193.811895,3296.610765,...,3332.546798,1268.932057,2063.722666,1281.082412,281.324394,0.003248,540.429021,4.970936,0.266457,0.664790
std,4.241818,4.724715,1911.414225,1910.020462,2064.795406,2314.219498,1944.307695,1891.118720,1898.716599,1864.218665,...,1867.600005,1195.428854,1151.422925,830.201350,13.854246,0.003646,610.147342,6.697951,0.199619,0.472068
min,-57.171004,-74.840663,-56.392936,299.238220,308.116962,136.928851,169.569203,84.967215,88.123196,13.675767,...,190.315943,30.882181,49.438014,47.248169,240.090890,0.000091,-119.033169,-0.163469,-0.643869,0.000000
25%,-13.857931,-20.492101,937.234240,936.750310,956.722972,1226.282068,1790.692556,1898.828877,1961.156403,2055.879131,...,2080.958363,550.752852,1294.068564,745.015448,270.741941,0.001020,129.433855,1.324655,0.102681,0.000000
50%,-11.442431,-17.516489,1142.971343,1185.170514,1399.391762,1614.316060,2296.427619,2526.479505,2598.806506,2766.040147,...,2832.929006,789.932894,1881.670377,1042.564710,282.309243,0.001846,305.315006,2.572516,0.249438,1.000000
75%,-9.245148,-14.824632,1962.460051,2061.126093,2461.075116,2802.601457,3276.227447,3660.143043,3735.164494,3942.121719,...,4019.123092,1400.731515,2623.288238,1614.847955,294.381850,0.003916,792.012585,5.339283,0.417680,1.000000
max,14.057914,7.074079,13624.663133,14274.620618,15724.538745,18868.088113,17166.621988,16868.053089,17286.252694,16836.840939,...,19489.963954,13943.846879,13135.748022,11376.935402,305.419884,0.045823,5376.089584,73.877508,0.796425,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62000 entries, 0 to 61999
Columns: 217 entries, S2_B2_jan to LABELS
dtypes: float64(217)
memory usage: 102.6 MB


In [6]:
df.dtypes

S2_B2_jan                       float64
S2_B3_jan                       float64
S2_B4_jan                       float64
S2_B5_jan                       float64
S2_B6_jan                       float64
                                 ...   
ERA5_total_precipitation_dec    float64
topo_elevation_dec              float64
topo_slope_dec                  float64
NDVI_dec                        float64
LABELS                          float64
Length: 217, dtype: object

In [7]:
labels = df[["LABELS"]]

In [8]:
labels.value_counts()

LABELS
1.0       41217
0.0       20783
dtype: int64

In [9]:
# There is some imbalance in the data. But not much.

In [10]:
df.isna().sum()

S2_B2_jan                       0
S2_B3_jan                       0
S2_B4_jan                       0
S2_B5_jan                       0
S2_B6_jan                       0
                               ..
ERA5_total_precipitation_dec    0
topo_elevation_dec              0
topo_slope_dec                  0
NDVI_dec                        0
LABELS                          0
Length: 217, dtype: int64

# Handling Duplicates:

In [11]:
#Droppping duplicates
df = df.drop_duplicates()

# Removing values which have the same features but different labels:

In [22]:
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0.0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1.0
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1.0
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1.0
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0.0
61996,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1.0
61997,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0.0
61998,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1.0


In [23]:
df_limited = df.drop(columns=["LABELS"])

In [24]:
df_limited

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
61996,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
61997,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
61998,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [25]:
df_limited = df_limited.drop_duplicates()
df_limited

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
61996,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
61997,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
61998,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [27]:
df = df.loc[df_limited.index]

In [28]:
df.to_csv(r"./new_train.csv")